# Model Zoo

In this notebook you will see every kind of model in AutoProf. Many models have unique capabilities and features, this will be introduced here, though fully taking advantage of them will be dependent on your science case.

For a family tree of all the AutoProf models see: link

Note, we will not be covering Compound_Model or Batch_Model here as those require a dedicated discussion. See the notebook: CombinedModels for that introduction.

In [ ]:
import autoprof as ap
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

## Sky Models

### Flat Sky Model

In [ ]:
M = ap.models.AutoProf_Model(name = "flat sky", model_type = "flat sky model", parameters = {"center":[50,50], "sky": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(figsize = (7,6))
ap.plots.model_image(fig, ax, M)
plt.show()

### Plane Sky Model

In [ ]:
M = ap.models.AutoProf_Model(name = "plane sky", model_type = "plane sky model", parameters = {"center":[50,50], "sky": 10, "delta": [1e-2,2e-2]})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(figsize = (7,6))
ap.plots.model_image(fig, ax, M)
plt.show()

## Star Models

### PSF Star

In [ ]:
from scipy.special import jv
xx, yy = np.meshgrid(np.linspace(-49,49,99), np.linspace(-49,49,99))
x = np.sqrt(xx**2 + yy**2)/5
PSF = (2*jv(1, x)/x)**2 + 1e-4 # the PSF can be any image, here we construct an airy disk
target = ap.image.Target_Image(data = np.zeros((100,100)), pixelscale = 1, psf = PSF) # the target image holds the PSF for itself

M = ap.models.AutoProf_Model(name = "psf star", model_type = "psf star model", target = target, parameters = {"center": [50,50], "sky": 1})
print(M.parameter_order)

fig, ax = plt.subplots(1,2, figsize = (14,6))
ax[0].imshow(np.log10(target.psf.detach().cpu().numpy()), origin = "lower")
x = np.linspace(-49,49,99)/5
ax[1].plot(x, np.log10((2*jv(1, x)/x)**2))
plt.show()

### Gaussian Star

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian star", model_type = "gaussian star model", parameters = {"center":[50,50], "sigma": 10, "flux": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ax[1].plot(np.linspace(0,50,100), M.radial_model(torch.linspace(0,50,100)).detach().cpu().numpy())
plt.show()

## Galaxy Models

### NonParametric Galaxy Model

In [ ]:
x = np.linspace(0,10,19)
nonparametric_profile = np.sin(x*2+2)/20 + 1 - x/20
M = ap.models.AutoProf_Model(name = "nonparametric galaxy", model_type = "nonparametric galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic galaxy", model_type = "sersic galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential galaxy", model_type = "exponential galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Galaxy Model

In [ ]:
M = ap.models.AutoProf_Model(name = "Gaussian", model_type = "gaussian galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric superellipse", model_type = "nonparametric superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic superellipse", model_type = "sersic superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential superellipse", model_type = "exponential superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian SuperEllipse

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian superellipse", model_type = "gaussian superellipse galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "C0": 2, "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric Fourier

In [ ]:
fourier_am = np.array([0.1, 0.3, -0.2])
fourier_phim = np.array([10*np.pi/180, 0, 40*np.pi/180])
M = ap.models.AutoProf_Model(name = "nonparametric fourier", model_type = "nonparametric fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic fourier", model_type = "sersic fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential fourier", model_type = "exponential fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Fourier

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian fourier", model_type = "gaussian fourier galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "am": fourier_am, "phim": fourier_phim, "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric warp", model_type = "nonparametric warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic warp", model_type = "sersic warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exp warp", model_type = "exponential warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gauss warp", model_type = "gaussian warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric super warp", model_type = "nonparametric superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "C0": 2, "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic super warp", model_type = "sersic superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "C0": 2, "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential super warp", model_type = "exponential superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "C0": 2, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gaussian SuperEllipse Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gauss super warp", model_type = "gaussian superellipse warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "C0": 2, "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric Fourier Warp

not sure how this abomination would fit a galaxy, but you are welcome to try

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric fourier warp", model_type = "nonparametric fourier warp galaxy model", modes = (1,3,4), parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "am": fourier_am, "phim": fourier_phim, "I(R)": nonparametric_profile})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Sersic Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic fourier warp", model_type = "sersic fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "am": fourier_am, "phim": fourier_phim, "n": 3, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Exponential Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "exp fourier warp", model_type = "exponential fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "am": fourier_am, "phim": fourier_phim, "Re": 10, "Ie": 1})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### Gassian Fourier Warp

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian fourier warp", model_type = "gaussian fourier warp galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "q(R)": np.linspace(0.2,0.4,19), "PA(R)": np.linspace(0,np.pi-0.1,19), "am": fourier_am, "phim": fourier_phim, "sigma": 5, "flux": 100})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.galaxy_light_profile(fig,ax[1],M)
plt.show()

### NonParametric Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "nonparametric ray", model_type = "nonparametric ray galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "I(R)": [nonparametric_profile, nonparametric_profile*2]})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Sersic Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "sersic ray", model_type = "sersic ray galaxy model", rays = 2, parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": [1,3], "Re": [10,5],"Ie": [1,0.5]})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Exponential Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "exponential ray", model_type = "exponential ray galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "Re": [10,5], "Ie": [1,2]})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()

### Gaussian Ray

In [ ]:
M = ap.models.AutoProf_Model(name = "gaussian ray", model_type = "gaussian ray galaxy model", parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "sigma": [5,8], "flux": [100,150]})
print(M.parameter_order)
M.sample()
fig, ax = plt.subplots(1,2, figsize = (14,6))
ap.plots.model_image(fig, ax[0], M)
ap.plots.ray_light_profile(fig,ax[1],M)
plt.show()